In [ ]:
import numpy as np
import scipy.integrate as spi
import matplotlib.pyplot as plt
%matplotlib inline

contributePressure ( int i, float3 p, int cell, float &sum_p6k ){
    if ( fbuf.bufI(FGRIDCNT)[cell] == 0 ) return 0.0;                       
    float3 dist;
    float dsq, r, q, b, c, sum = 0.0;
    register float d2 = fparam.psimscale * fparam.psimscale;                
    register float r2 = fparam.r2 / d2;                                     
    register float H  = fparam.H;                                           
    register float sr = fparam.psmoothradius;
    int clast = fbuf.bufI(FGRIDOFF)[cell] + fbuf.bufI(FGRIDCNT)[cell];      
    uint k=0;
    for ( int cndx = fbuf.bufI(FGRIDOFF)[cell]; cndx < clast; cndx++ ) {    
        k++;
        int pndx = fbuf.bufI(FGRID) [cndx];                                 
        dist = p - fbuf.bufF3(FPOS) [pndx];                                 
        dsq = (dist.x*dist.x + dist.y*dist.y + dist.z*dist.z);              
        if ( dsq < r2 && dsq > 0.0) {              
            r=sqrt(dsq);
            q=r/sr;                                                         
            b=(1-q);                     
            b*=b; 
            b*=b;                        
            sum  += b*(4*q +1);                                 
        }
    }
    return sum;                                                             
}

computePressure ( int pnum ){
    uint i = __mul24(blockIdx.x, blockDim.x) + threadIdx.x; 
    if ( i >= pnum ) return;
    int nadj = (1*fparam.gridRes.z + 1)*fparam.gridRes.x + 1;
    uint gc = fbuf.bufI(FGCELL) [i];
    if ( gc == GRID_UNDEF ) return;
    gc -= nadj;
    float3 pos = fbuf.bufF3(FPOS) [i];
    float sum = 0.0;
    for (int c=0; c < fparam.gridAdjCnt; c++) { sum += contributePressure ( i, pos, gc + fparam.gridAdj[c], sum_p6k ); }
    sum = sum * fparam.pmass * fparam.wendlandC2kern;
    if ( sum == 0.0 ) sum = 1.0;
    fbuf.bufF(FPRESS)  [ i ] = ( sum - fparam.prest_dens ) * fparam.pintstiff;   
    fbuf.bufF(FDENSITY)[ i ] = 1.0f  / sum;    
}

contribForce( int i, float3 ipos, float3 iveleval, float ipress, float idens, int cell){
    if ( fbuf.bufI(FGRIDCNT)[cell] == 0 ) return make_float3(0,0,0);
    float  dsq, sdist, c, r, sr=fparam.psmoothradius;
    float3 pterm= make_float3(0,0,0), sterm= make_float3(0,0,0), vterm= make_float3(0,0,0), forcej= make_float3(0,0,0), 
           delta_v= make_float3(0,0,0);
    float3 dist = make_float3(0,0,0),      eterm = make_float3(0,0,0),    force = make_float3(0,0,0);
    uint   j;
    int    clast    = fbuf.bufI(FGRIDOFF)[cell] + fbuf.bufI(FGRIDCNT)[cell];
    uint k =0 ;
    for (int cndx = fbuf.bufI(FGRIDOFF)[cell]; cndx < clast; cndx++ ) {
        k++;
        j           = fbuf.bufI(FGRID)[ cndx ];
        dist        = ( ipos - fbuf.bufF3(FPOS)[ j ] );
        dsq         = (dist.x*dist.x + dist.y*dist.y + dist.z*dist.z);
        r           = sqrt(dsq);
        if ( dsq < 1 /*fparam.rd2*/ && dsq > 0) {   
                float kern = pow((sr - r),3);
                sdist   = sqrt(dsq * fparam.d2);
                float press = 100*(ipress+fbuf.bufF(FPRESS)[j]);
                pterm = idens * fbuf.bufF(FDENSITY)[j] *  100.0* (dist/r) *(press*kern - (fparam.psurface_t/*0.4*/)*pow((sr - r),2));
                delta_v = fbuf.bufF3(FVEVAL)[j] - iveleval;
                vterm =  100000.0* delta_v * kern;
                force   +=  pterm + vterm  ;
        }
    return force;
    }                                                                                                                          
}
      
computeForce ( int pnum, bool freeze, uint frame){
    uint i = __mul24(blockIdx.x, blockDim.x) + threadIdx.x; 
    if ( i >= pnum ) return;
    uint gc = fbuf.bufI(FGCELL)[ i ];
    if ( gc == GRID_UNDEF ) return;
    gc -= (1*fparam.gridRes.z + 1)*fparam.gridRes.x + 1;
    register float3 force, eterm, dist;
    force = make_float3(0,0,0);    eterm = make_float3(0,0,0);     dist  = make_float3(0,0,0);
    float dsq, abs_dist;
    uint bondsToFill = 0;
    uint bonds[BONDS_PER_PARTICLE][2];
    float bond_dsq[BONDS_PER_PARTICLE];
    for (int a=0; a<BONDS_PER_PARTICLE;a++) {
        bonds[a][0]= UINT_MAX;
        bonds[a][1]= UINT_MAX;
        bond_dsq[a]= fparam.rd2;
    } 
    uint i_ID = fbuf.bufI(FPARTICLE_ID)[i];
    float3  pvel = {fbuf.bufF3(FVEVAL)[ i ].x,  fbuf.bufF3(FVEVAL)[ i ].y,  fbuf.bufF3(FVEVAL)[ i ].z};
    bool hide;
    bool long_bonds = false;
    for (int a=0;a<BONDS_PER_PARTICLE;a++){
        uint bond               = i*BOND_DATA + a*DATA_PER_BOND;
        uint j                  = fbuf.bufI(FELASTIDX)[bond];
        float restlength        = fbuf.bufF(FELASTIDX)[bond + 2];
        float elastic_limit     = fbuf.bufF(FELASTIDX)[bond + 1];
        float modulus           = fbuf.bufF(FELASTIDX)[bond + 3];
        float damping_coeff     = fbuf.bufF(FELASTIDX)[bond + 4];
        uint  other_particle_ID = fbuf.bufI(FELASTIDX)[bond + 5];
        uint  bondIndex         = fbuf.bufI(FELASTIDX)[bond + 6];
        float3 j_pos    = make_float3(fbuf.bufF3(FPOS)[ j ].x,  fbuf.bufF3(FPOS)[ j ].y,  fbuf.bufF3(FPOS)[ j ].z); 
        dist            = ( fbuf.bufF3(FPOS)[ i ] - j_pos  );             
        dsq             = (dist.x*dist.x + dist.y*dist.y + dist.z*dist.z);
        abs_dist        = sqrt(dsq) + FLT_MIN;                
        float3 rel_vel  = fbuf.bufF3(FVEVAL)[ j ] - pvel;     
        float spring_strain = fmaxf(0.0, (abs_dist-restlength)/restlength);
                                           
        fbuf.bufF(FELASTIDX)[bond + /*6*/strain_sq_integrator] 
                                           = (fbuf.bufF(FELASTIDX)[bond + /*6*/strain_sq_integrator] + spring_strain*spring_strain);  
        fbuf.bufF(FELASTIDX)[bond + /*7*/strain_integrator] 
                                           = (fbuf.bufF(FELASTIDX)[bond + /*7*/strain_integrator] + spring_strain);
        if(!(eterm.x!=eterm.x||eterm.y!=eterm.y||eterm.z!=eterm.z) ) force -= eterm;
                atomicAdd( &fbuf.bufF3(FFORCE)[ j ].x, eterm.x);                    
                atomicAdd( &fbuf.bufF3(FFORCE)[ j ].y, eterm.y);
                atomicAdd( &fbuf.bufF3(FFORCE)[ j ].z, eterm.z);                    
        }
        if (abs_dist >= elastic_limit){                                         
                fbuf.bufF(FELASTIDX)[i*BOND_DATA + a*DATA_PER_BOND +2]=0.0;
                uint bondIndex_ = fbuf.bufI(FELASTIDX)[i*BOND_DATA + a*DATA_PER_BOND +6];
                bondsToFill++;
        }
    }
    float3 fluid_force_sum = make_float3(0,0,0);
    for (int c=0; c < fparam.gridAdjCnt; c++) {                                     
        float3 fluid_force = make_float3(0,0,0);
        fluid_force = contributeForce ( i, fbuf.bufF3(FPOS)[ i ], fbuf.bufF3(FVEVAL)[ i ], fbuf.bufF(FPRESS)[ i ], 
                                       fbuf.bufF(FDENSITY)[ i ], gc + fparam.gridAdj[c]);
        fluid_force_sum += fluid_force;
    }
    force += fluid_force_sum * fparam.pmass ; 
    atomicAdd(&fbuf.bufF3(FFORCE)[ i ].x, force.x);                                 
    atomicAdd(&fbuf.bufF3(FFORCE)[ i ].y, force.y);                                
    atomicAdd(&fbuf.bufF3(FFORCE)[ i ].z, force.z);                                 
} 
